![logo](./img/LogoLine_horizon_C3S.png)

<br>

# Seasonal forecast (hindcast) preprocessing

### About

<br>

### Install packages

In [6]:
# Miscellaneous operating system interfaces
import os

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime


# 1 - Process seasonal hindcast grib files (24h, total precipitation)

<hr>

In [7]:
# Define the coordinates to interpolate to
#interp_longitude=2.25 #C1=C2=2.25  Not necessary (2 coordinates < SEAS5 resolution of 1ª)
#interp_latitude=42.3  #C1=42.3, C2=42.05  Not necessary (2 coordinates < SEAS5 resolution of 1ª)
variable_name = 'total_precipitation'
var_short = 'tp'

Here we specify a data directory in which the grib files are located

In [8]:
DATADIR = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal'

Obtaining daily precipitation data sets

In [8]:
#list to store results
hind_all = []

#for loop for all years (1993-2016)
for time in range(1993,2017):
    
    #for loop for all months (1-12)
    for month_num in range(1,13):

        #open file for the corresponding month
        hind = xr.open_dataset(f'{DATADIR}/ecmwf_seas5_1993-2016_'+ str(month_num) + '_hindcast_daily_' + var_short + '.grib', engine='cfgrib')
        hind_var = hind[var_short]

        # Interpolate the values for the specified coordinates
        hind_interp = hind_var.interp(latitude=interp_latitude, longitude=interp_longitude, method="nearest")

        #create the subset for the specific year and month
        year_subset = f'{time}-{month_num:02d}-01T00:00:00.000000000'
        subset_hind = hind_interp.sel(time=year_subset)
        
        #verify number of members
        num_members = len(subset_hind)

        #calculate the difference throughout the steps (215 days) to obtain daily precipitation
        subset_hind_diff = subset_hind.diff(dim='step') # by default does it along the last axis

        # Prepend the value of the original step 1 to the differences array
        first_step_value = subset_hind.isel(step=0)  # Get the value of the original step 1

        # Concatenate first_step_value with subset_hind_diff along the 'step' dimension in order to obtain the 215 steps
        subset_hind_diff_with_first = xr.concat([first_step_value, subset_hind_diff], dim='step')
        subset_hind_diff_with_first

        #add results to the list
        hind_all.append(subset_hind_diff_with_first)

Save data to npy file (one per subcatchment C1, C2)

In [11]:
from copy import deepcopy
hind_all_tp = np.empty(1,dtype=object)
hind_all_tp[0] = deepcopy(hind_all)
np.save('hind_all_tp_C1',hind_all_tp) #change _C1 or C2 accordingly.

# 2. Export seasonal hindcast csv files (daily precipitation)

Import npy file to create hind files for each year, month and member

In [13]:
hind_file = f"{DATADIR}/hindcast/hind_npy_files/hind_all_{var_short}_C2.npy"  #C1 and C2 are the same for SEAS5 because of the low resolution
hind_all = (np.load(hind_file,allow_pickle=True))[0] 
hind_all

[<xarray.DataArray 'tp' (number: 25, step: 215)> Size: 22kB
 array([[0.        , 0.        , 0.00052643, ..., 0.        , 0.        ,
         0.00012207],
        [0.        , 0.        , 0.00041199, ..., 0.0012207 , 0.00012207,
         0.00012207],
        [0.        , 0.        , 0.00135803, ..., 0.00146484, 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.00196075, ..., 0.        , 0.00024414,
         0.        ],
        [0.        , 0.        , 0.00065613, ..., 0.        , 0.01818848,
         0.0012207 ],
        [0.        , 0.        , 0.00016785, ..., 0.00048828, 0.        ,
         0.00012207]], dtype=float32)
 Coordinates:
   * number      (number) int32 100B 0 1 2 3 4 5 6 7 ... 17 18 19 20 21 22 23 24
     time        datetime64[ns] 8B 1993-01-01
   * step        (step) timedelta64[ns] 2kB 1 days 2 days ... 214 days 215 days
     surface     float64 8B 0.0
     valid_time  (step) datetime64[ns] 2kB 1993-01-02 1993-01-03 ... 1993-08-04
  

In [10]:
#exploring npy file
print(type(hind_all))
print(len(hind_all))

<class 'list'>
288


In [14]:
# Set path to save hindcast files
save_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast'

In [15]:
# Define a function to save each sub-array
def save_sub_array(dataarray, number, list_idx, save_path):
    
    # Extract the sub-array
    sub_array = dataarray.sel(number=number).values
        
    # Convert the precipitation from m to mm
    sub_array_mm = sub_array*1000
        
    # Define the filename (25 members, 288 times(24 years X 12 months))
    filename = os.path.join(save_path,f"hindcast_member{number}_time{list_idx}_{var_short}.csv")
    
    # Save the sub-array to a .csv file
    np.savetxt(filename, sub_array_mm) #, delimiter=',', fmt='%.6f')
    #print(f"Saved {filename}")
    
# Iterate through the list
for list_idx, dataarray in enumerate(hind_all):
    # Ensure the 'number' coordinate is numeric
    if isinstance(dataarray['number'].values, np.ndarray):
        numbers = dataarray['number'].values
    else:
        numbers = np.arange(len(dataarray['number']))
    
    # Iterate through each 'number'
    for number in numbers:
        save_sub_array(dataarray, number, list_idx, save_path)


Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member11_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member12_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member13_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member14_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member15_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member16_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member17_time11.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member3_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member4_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member5_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member6_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member7_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member8_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member9_time16.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcas

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member7_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member8_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member9_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member10_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member11_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member12_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member13_time22.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hin

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member16_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member17_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member18_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member19_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member20_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member21_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member22_time28.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member15_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member16_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member17_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member18_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member19_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member20_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member21_time32.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/

...

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member0_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member1_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member2_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member3_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member4_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member5_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member6_time266.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member5_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member6_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member7_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member8_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member9_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member10_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member11_time271.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasona

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member10_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member11_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member12_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member13_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member14_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member15_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member16_time274.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/se

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member9_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member10_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member11_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member12_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member13_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member14_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member15_time280.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/sea

Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member20_time285.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member21_time285.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member22_time285.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member23_time285.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member24_time285.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member0_time286.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast\hind_C2_tp_member1_time286.csv
Saved C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seas